In [ ]:
import os
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np

# Funtion to load and preprocess data

In [ ]:
def load_crop_data(path='data/frontal'):

    res_size = 512

    res = []

    for filename in os.listdir(path):
        image_path = os.path.join(path, filename)
        image = Image.open(image_path)
        width, height = image.size
        min_side = min(width, height)
        p1 = (width - min_side) // 2
        p2 = (height - min_side) // 2
        p3 = (width + min_side) // 2
        p4 = (height + min_side) // 2
        cropped_image = image.crop((p1, p2, p3, p4))
        cropped_image = cropped_image.resize((res_size, res_size))
        image_array = np.array(cropped_image)
        res.append(image_array[np.newaxis, :, :, :])

    res = np.concatenate(res, axis=0)
    return res

# Function to display generated image

In [ ]:
def display_image(epoch_idx):
    return Image.open(f"generated/generated_images_epoch_{epoch_idx + 1}.png")

# Defining generator structure

In [ ]:
def build_generator(latent_dim):
    model = keras.Sequential()
    model.add(layers.Dense(8 * 8 * 64, input_dim=latent_dim))
    model.add(layers.Reshape((8, 8, 64)))
    model.add(layers.Conv2DTranspose(64, kernel_size=8, strides=2, padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Conv2DTranspose(32, kernel_size=8, strides=2, padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Conv2DTranspose(16, kernel_size=4, strides=2, padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Conv2DTranspose(8, kernel_size=4, strides=2, padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Conv2DTranspose(8, kernel_size=4, strides=2, padding="same"))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Conv2DTranspose(3, kernel_size=4, strides=2, padding="same", activation="sigmoid"))
    return model

# Defining discriminator structure

In [ ]:
def build_discriminator():
    model = keras.Sequential()
    model.add(layers.Conv2D(8, kernel_size=4, strides=2, padding="same", input_shape=(512, 512, 3)))
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Conv2D(16, kernel_size=4, strides=2, padding="same"))
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Conv2D(32, kernel_size=4, strides=4, padding="same"))
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Conv2D(64, kernel_size=4, strides=2, padding="same"))
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Conv2D(128, kernel_size=4, strides=4, padding="same"))
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation="sigmoid"))
    return model

# Defining losses

In [ ]:
cross_entropy = keras.losses.BinaryCrossentropy(from_logits=True)

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

# Model initialization

In [ ]:
latent_dim = 16384
generator = build_generator(latent_dim)
discriminator = build_discriminator()

generator_optimizer = keras.optimizers.Adam(learning_rate=0.00002, beta_1=0.2)
discriminator_optimizer = keras.optimizers.Adam(learning_rate=0.00002, beta_1=0.2)

# Function to generate an image to observe the trainig process

In [ ]:
def generate_images(model, epoch, input):
    predictions = model(input, training=False)

    fig = plt.figure(figsize=(4, 4))
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i + 1)
        plt.imshow(predictions[i, :, :, :])
        plt.axis('off')

    plt.savefig(f"generated/generated_images_epoch_{epoch + 1}.png")
    plt.show()
    plt.clf()

# Defining the training step

In [ ]:
@tf.function
def train_step(images):

    noise = tf.random.normal([batch_size, latent_dim])


    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)


        real_output = discriminator(images, training=True)

        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

# Start training

In [ ]:
batch_size = 32

X_train = load_crop_data()
X_train = (X_train.astype('float32')) / 255
train_dataset = tf.data.Dataset.from_tensor_slices(X_train).shuffle(X_train.shape[0]).batch(batch_size)

epochs = 10000
for epoch in range(epochs):
    for image_batch in train_dataset:
        train_step(image_batch)

    if (epoch + 1) % 10 == 0:
        generate_images(generator, epoch, tf.random.normal([16, latent_dim]))
        print(f"Epoch {epoch + 1} completed.")

display_image(epochs)